In [3]:
import gzip
import unicodedata
import multiprocessing as mp

import nltk
from spacy.lang.pl import Polish
from spacy.tokenizer import Tokenizer

In [4]:
with gzip.open("../data/cytaty.txt.gz", "rt") as f:
    txt = f.read()

In [27]:
txt = txt.replace("&lt;br", "")
txt = txt.replace("/&gt;", "")

In [38]:
def word_tokenize(word):
    return nltk.word_tokenize(word, "polish")
    
with mp.Pool(6) as p:
    nltk_tokenized = p.map(word_tokenize, txt.split("\n*"))

In [ ]:
pl = Polish()
tokenizer = Tokenizer(pl.vocab)

with mp.Pool(6) as p:
    spacy_tokenized = p.map(tokenizer, txt.split("\n*"))

In [39]:
def lstrip(word: str):
    if word and unicodedata.category(word[0]).startswith("P"):
        return lstrip(word[1:])
    return word

def rstrip(word: str):
    if word and unicodedata.category(word[-1]).startswith("P"):
        return rstrip(word[:-1])
    return word

def strip(word: str):
    return lstrip(rstrip(word))

def tokenize(sentence):
    return list(filter(None, map(strip, sentence.split(" "))))

In [75]:
def strip(word: str):
    idxs = [
        idx
        for c, idx in zip(map(unicodedata.category, word), range(len(word)))
        if not c.startswith("P")
    ]
    return word[min(idxs, default=0):max(idxs, default=len(word)) + 1]

def tokenize(sentence: str):
    return list(filter(None, map(strip, sentence.split(" "))))

In [77]:
with mp.Pool(6) as p:
    tokenized = p.map(tokenize, txt.split("\n*"))

In [78]:
with open("tokens.txt", "wt") as f:
    for nltk_tokens, our_tokens, spacy_tokens in list(zip(nltk_tokenized, tokenized, spacy_tokenized)):
        if nltk_tokens != our_tokens:
            f.write("NLTK: " + str(nltk_tokens) + "\n")
            f.write("NASZ: " + str(our_tokens) + "\n")
            f.write("SPACY:" + str(list(map(str, spacy_tokens))) + "\n")
            f.write("*" * 30 + "\n")